
Capstone Project - The Battle of Neighborhoods (Week 1-2)
Background

Sydney is Australia’s oldest city, meaning centuries of the country’s history have unfolded in its streets. Every city has its own unique feel and vibe, which is determined by a number of things. The local historic sites are no doubt one of the largest contributing factors to the aura that surrounds a city. In this project we explore, segment, and cluster the historic sites in the city of Sydney. We believe this will help tourists to make their onwn travel plan, and therefore save them a lot of time.

 
Business Problem

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this problem, we are going to cluster sydney historic sites in order to help the tourists.  

Data section

Data on Sydney historic sites was extracted from the wiki website (https://en.wikipedia.org/wiki/List_of_heritage_houses_in_Sydney).  

To explore the venues close to each site, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. 

 Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Historic site Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

1. Collect Historic site Data¶

After importing the necessary libraries, we download the data from the Wiki website as follows:


In [1]:
# Import necessary library
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [158]:
#url = "https://www.educationstandards.nsw.edu.au/wps/portal/nesa/11-12/hsc/results-certificates/merit-lists/first-in-course"
from bs4 import BeautifulSoup
import requests
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_heritage_houses_in_Sydney"
source = requests.get(url).text

In [3]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [4]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['House','Year','Neighborhood','CHL','','','','']
df = pd.DataFrame(columns = column_names)
df
column_names = ['House','Year','Neighborhood','CHL']
df1 = pd.DataFrame(columns = column_names)
df1

,House,Year,Neighborhood,CHL


In [5]:
for tr in table.find_all('tr'):
    neighborhoods=[]
    for td in tr.find_all('td'):
        neighborhoods.append(td.text.strip())
        #bb.loc[len(bb)]=neighborhoods
    if len(neighborhoods)>4:
        df.loc[len(df)] = neighborhoods

In [6]:
for tr in table.find_all('tr'):
    neighborhoods=[]
    for td in tr.find_all('td'):
        neighborhoods.append(td.text.strip())
        #bb.loc[len(bb)]=neighborhoods
    if len(neighborhoods)==4:
        df1.loc[len(df1)] = neighborhoods

In [7]:
df=df[['House','Year','Neighborhood']]
df1=df1[['House','Year','Neighborhood']]
frames = [df, df1]
df = pd.concat(frames)
df['House'] =  df['House'].astype(str) + ', Sydney' 
df.head()

,House,Year,Neighborhood
0,"Elizabeth Farm, Sydney",1790s,Parramatta
1,"Vaucluse House, Sydney",1839,Vaucluse
2,"Government House, Sydney",1830s,Sydney
3,"Admiralty House, Sydney",1840s,Kirribilli
4,"Bronte House, Sydney",1845,Bronte


In [8]:
#from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [9]:
for index, item in df.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.House}")

index: 0
item: House           Elizabeth Farm, Sydney
Year                             1790s
Neighborhood                Parramatta
Name: 0, dtype: object
item.Street only: Elizabeth Farm, Sydney
index: 1
item: House           Vaucluse House, Sydney
Year                              1839
Neighborhood                  Vaucluse
Name: 1, dtype: object
item.Street only: Vaucluse House, Sydney
index: 2
item: House           Government House, Sydney
Year                               1830s
Neighborhood                      Sydney
Name: 2, dtype: object
item.Street only: Government House, Sydney
index: 3
item: House           Admiralty House, Sydney
Year                              1840s
Neighborhood                 Kirribilli
Name: 3, dtype: object
item.Street only: Admiralty House, Sydney
index: 4
item: House           Bronte House, Sydney
Year                            1845
Neighborhood                  Bronte
Name: 4, dtype: object
item.Street only: Bronte House, Sydney
index: 5
item: H

In [10]:
geolocator = Nominatim(user_agent="foursquare_agent")

In [11]:
df=df.reset_index()
df=df.drop(columns=['index'])

In [12]:
df['Latitude']=0
df['Longitude']=0
df.index
#location = geolocator.geocode(df.iloc[44]['House'])
#if location!=None:
#    df.iloc[44]['Latitude'] = location.latitude
#    df.iloc[44]['Longitude'] = location.longitude
#    df['Latitude']= location.latitude
#    df['Longitude']= location.longitude
#df.iloc[1]['Latitude']

df = df.applymap(str)


,House,Year,Neighborhood,Latitude,Longitude
0,"Elizabeth Farm, Sydney",1790s,Parramatta,0,0
1,"Vaucluse House, Sydney",1839,Vaucluse,0,0
2,"Government House, Sydney",1830s,Sydney,0,0
3,"Admiralty House, Sydney",1840s,Kirribilli,0,0
4,"Bronte House, Sydney",1845,Bronte,0,0
5,"The Swifts, Sydney",1870s,Darling Point,0,0
6,"Berith Park, Sydney",c. 1909,Wahroonga,0,0
7,"Experiment Farm Cottage, Sydney",1790s,Harris Park,0,0
8,"Old Government House, Sydney",1790s,Parramatta,0,0
9,"Cadmans Cottage, Sydney",1810s,The Rocks,0,0


In [15]:
df['Latitude']=df['Latitude'].astype('float')
df['Longitude']=df['Longitude'].astype('float')
df = df[df['Latitude']!=0]
df.head()

,House,Year,Neighborhood,Latitude,Longitude
0,"Elizabeth Farm, Sydney",1790s,Parramatta,-33.821191,151.017958
1,"Vaucluse House, Sydney",1839,Vaucluse,-33.855789,151.273152
2,"Government House, Sydney",1830s,Sydney,-33.859925,151.215125
3,"Admiralty House, Sydney",1840s,Kirribilli,-33.851728,151.218344
4,"Bronte House, Sydney",1845,Bronte,-33.904213,151.263777
5,"The Swifts, Sydney",1870s,Darling Point,-33.869513,151.238205
6,"Berith Park, Sydney",c. 1909,Wahroonga,-33.817134,150.957947
7,"Experiment Farm Cottage, Sydney",1790s,Harris Park,-33.820176,151.012583
8,"Old Government House, Sydney",1790s,Parramatta,-33.812092,150.997307
9,"Cadmans Cottage, Sydney",1810s,The Rocks,-33.858891,151.209187


2. Explore and Understand Data

In [43]:
address = 'Sydney'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney are -33.8548157, 151.2164539.


In [44]:
# create map of New York using latitude and longitude values
map_Sydney = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['House'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Sydney)  
    
map_Sydney

3. Data preparation and preprocessing

In [45]:
CLIENT_ID = 'OMMFL1P4K5B5ON3H1IC1ZV32ALIBPZGCMIFCZFK5SF42WU3M' # your Foursquare ID
CLIENT_SECRET = 'M1WJYMCHCHVZA3XRXZSFD4NTFZO3Q1O4D52HCYQSPOAC4MF5' # your Foursquare Secret
VERSION = '20180605'

In [48]:
neighborhood_latitude = df.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Vaucluse are -33.8557893, 151.27315161103604.


In [47]:
df.loc[1, 'Latitude']

-33.8557893

In [49]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url  



'https://api.foursquare.com/v2/venues/explore?&client_id=OMMFL1P4K5B5ON3H1IC1ZV32ALIBPZGCMIFCZFK5SF42WU3M&client_secret=M1WJYMCHCHVZA3XRXZSFD4NTFZO3Q1O4D52HCYQSPOAC4MF5&v=20180605&ll=-33.8557893,151.27315161103604&radius=500&limit=100'

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8531a50f59680026d34741'},
 'response': {'headerLocation': 'Vaucluse',
  'headerFullLocation': 'Vaucluse, Sydney',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': -33.851289295499996,
    'lng': 151.27856029734767},
   'sw': {'lat': -33.8602893045, 'lng': 151.26774292472442}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b058765f964a5202f9022e3',
       'name': 'Vaucluse House',
       'location': {'address': 'Wentworth Road',
        'lat': -33.8547477724124,
        'lng': 151.27351751715287,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.8547477724124,
          'lng': 151.27351751715287}],
        'distance': 120,
        'postalCode': '2030',
     

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [64]:
venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Vaucluse House,History Museum,-33.854748,151.273518
1,Vaucluse Bay,Beach,-33.855507,151.274927
2,Vaucluse House Tearooms,Café,-33.855463,151.273096
3,Johnston's Lookout,Scenic Lookout,-33.859125,151.275374
4,Strickland House,Historic Site,-33.856243,151.268057


In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:
sydney_venues = getNearbyVenues(names=df['House'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Elizabeth Farm, Sydney
Vaucluse House, Sydney
Government House, Sydney
Admiralty House, Sydney
Bronte House, Sydney
The Swifts, Sydney
Berith Park, Sydney
Experiment Farm Cottage, Sydney
Old Government House, Sydney
Cadmans Cottage, Sydney
Juniper Hall, Sydney
Toxteth Park, Sydney
Glover cottages, Sydney
Camden Park, Sydney
Elizabeth Bay House, Sydney
Lindesay, Sydney
Newington House, Sydney
Rose Bay Cottage, Sydney
Darling House, Sydney
Carthona, Sydney
The Hermitage, Sydney
Richmond Villa, Sydney
Willandra, Sydney
Emu Hall, Sydney
Bishopscourt, Sydney
Fairfax House, Sydney
Greycliffe House, Sydney
Kirribilli House, Sydney
Stead House, Sydney
Strickland House, Sydney
Gladswood House, Sydney
Tresco, Sydney
Fernleigh Castle, Sydney
The Hermitage, Sydney
The Abbey, Sydney
Caerleon, Sydney
Rona, Sydney
St Helens Park House, Sydney
Woollahra House, Sydney
Bellevue, Sydney
The Crossways, Sydney
Highbury, Sydney
Boomerang, Sydney
Rose Seidler House, Sydney
Woolley House, Sydney


In [69]:
sydney_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Elizabeth Farm, Sydney",-33.821191,151.017958,Elizabeth Farm,-33.821050,151.017890,History Museum
1,"Elizabeth Farm, Sydney",-33.821191,151.017958,Rydges Hotel,-33.823620,151.021032,Hotel
2,"Elizabeth Farm, Sydney",-33.821191,151.017958,Rosehill Bowling Club,-33.819005,151.021345,Bowling Green
3,"Elizabeth Farm, Sydney",-33.821191,151.017958,Woolworths,-33.819395,151.020194,Supermarket
4,"Elizabeth Farm, Sydney",-33.821191,151.017958,Subway,-33.819585,151.019935,Sandwich Place


In [70]:
sydney_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Admiralty House, Sydney",8,8,8,8,8,8
"Bellevue, Sydney",42,42,42,42,42,42
"Berith Park, Sydney",4,4,4,4,4,4
"Bishopscourt, Sydney",2,2,2,2,2,2
"Bronte House, Sydney",18,18,18,18,18,18
"Cadmans Cottage, Sydney",100,100,100,100,100,100
"Caerleon, Sydney",15,15,15,15,15,15
"Camden Park, Sydney",8,8,8,8,8,8
"Carthona, Sydney",3,3,3,3,3,3


In [78]:
# one hot encoding
sydney_onehot = pd.get_dummies(sydney_venues[['Venue Category']], prefix="", prefix_sep="")
#sydney_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
sydney_onehot.insert(loc=0, column='Neighborhood', value=sydney_venues['Neighborhood'] )
sydney_onehot.shape

(879, 159)

In [80]:
sydney_grouped = sydney_onehot.groupby('Neighborhood').mean().reset_index()
sydney_grouped.head()

,Neighborhood,Airport,American Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,...,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,"Admiralty House, Sydney",0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,"Bellevue, Sydney",0.0,0.02381,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0
2,"Berith Park, Sydney",0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,"Bishopscourt, Sydney",0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,"Bronte House, Sydney",0.0,0.00000,0.0,0.0,0.0,0.055556,0.0,0.0,0.055556,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


4. Modeling

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [130]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sydney_grouped['Neighborhood']

for ind in np.arange(sydney_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sydney_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted 

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Admiralty House, Sydney",Boat or Ferry,Park,Harbor / Marina,Motel,Scenic Lookout,Pier,Café,Zoo,Fish & Chips Shop,Fast Food Restaurant
1,"Bellevue, Sydney",Café,Italian Restaurant,Bar,Pub,Australian Restaurant,Deli / Bodega,Cheese Shop,Coffee Shop,Restaurant,Convenience Store
2,"Berith Park, Sydney",Fast Food Restaurant,Bus Station,Chinese Restaurant,Zoo,Electronics Store,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Farmers Market
3,"Bishopscourt, Sydney",Shopping Mall,Park,Zoo,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
4,"Bronte House, Sydney",Café,Beach,Juice Bar,Convenience Store,Restaurant,Park,Fish & Chips Shop,Burger Joint,Breakfast Spot,Pool
5,"Cadmans Cottage, Sydney",Café,Hotel,Australian Restaurant,Pub,Italian Restaurant,Scenic Lookout,Cocktail Bar,Ice Cream Shop,Park,Museum
6,"Caerleon, Sydney",Café,Pub,Gas Station,Pizza Place,South American Restaurant,Martial Arts Dojo,Malay Restaurant,Bowling Alley,Kids Store,Thai Restaurant
7,"Camden Park, Sydney",Café,Supermarket,Fast Food Restaurant,Australian Restaurant,Pizza Place,Park,Zoo,Flea Market,Fish Market,Fish & Chips Shop
8,"Carthona, Sydney",Park,Boat or Ferry,Zoo,Food Truck,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
9,"Darling House, Sydney",Café,Hotel,Australian Restaurant,Italian Restaurant,Japanese Restaurant,French Restaurant,Bar,Breakfast Spot,Malay Restaurant,Pub


In [131]:
# set number of clusters
kclusters = 5

sydney_grouped_clustering = sydney_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sydney_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([1, 1, 2, 0, 1, 1, 1, 1, 0, 1], dtype=int32)

In [132]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Admiralty House, Sydney",Boat or Ferry,Park,Harbor / Marina,Motel,Scenic Lookout,Pier,Café,Zoo,Fish & Chips Shop,Fast Food Restaurant
1,1,"Bellevue, Sydney",Café,Italian Restaurant,Bar,Pub,Australian Restaurant,Deli / Bodega,Cheese Shop,Coffee Shop,Restaurant,Convenience Store
2,2,"Berith Park, Sydney",Fast Food Restaurant,Bus Station,Chinese Restaurant,Zoo,Electronics Store,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Farmers Market
3,0,"Bishopscourt, Sydney",Shopping Mall,Park,Zoo,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
4,1,"Bronte House, Sydney",Café,Beach,Juice Bar,Convenience Store,Restaurant,Park,Fish & Chips Shop,Burger Joint,Breakfast Spot,Pool
5,1,"Cadmans Cottage, Sydney",Café,Hotel,Australian Restaurant,Pub,Italian Restaurant,Scenic Lookout,Cocktail Bar,Ice Cream Shop,Park,Museum
6,1,"Caerleon, Sydney",Café,Pub,Gas Station,Pizza Place,South American Restaurant,Martial Arts Dojo,Malay Restaurant,Bowling Alley,Kids Store,Thai Restaurant
7,1,"Camden Park, Sydney",Café,Supermarket,Fast Food Restaurant,Australian Restaurant,Pizza Place,Park,Zoo,Flea Market,Fish Market,Fish & Chips Shop
8,0,"Carthona, Sydney",Park,Boat or Ferry,Zoo,Food Truck,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
9,1,"Darling House, Sydney",Café,Hotel,Australian Restaurant,Italian Restaurant,Japanese Restaurant,French Restaurant,Bar,Breakfast Spot,Malay Restaurant,Pub


In [134]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sydney_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sydney_merged = sydney_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='House')

sydney_merged.head()

,House,Year,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Elizabeth Farm, Sydney",1790s,Parramatta,-33.821191,151.017958,1.0,Hotel,Train Station,Gas Station,Pizza Place,Platform,Restaurant,Sandwich Place,Bowling Green,Supermarket,History Museum
1,"Vaucluse House, Sydney",1839,Vaucluse,-33.855789,151.273152,1.0,History Museum,Historic Site,Scenic Lookout,Café,Beach,Zoo,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,"Government House, Sydney",1830s,Sydney,-33.859925,151.215125,1.0,Café,Australian Restaurant,Hotel,Garden,Cocktail Bar,Harbor / Marina,Theater,Bar,Boat or Ferry,Concert Hall
3,"Admiralty House, Sydney",1840s,Kirribilli,-33.851728,151.218344,1.0,Boat or Ferry,Park,Harbor / Marina,Motel,Scenic Lookout,Pier,Café,Zoo,Fish & Chips Shop,Fast Food Restaurant
4,"Bronte House, Sydney",1845,Bronte,-33.904213,151.263777,1.0,Café,Beach,Juice Bar,Convenience Store,Restaurant,Park,Fish & Chips Shop,Burger Joint,Breakfast Spot,Pool


In [149]:
sydney_merged.dropna(subset = ["Cluster Labels"], inplace=True)
sydney_merged["Cluster Labels"] = pd.to_numeric(sydney_merged["Cluster Labels"],downcast='integer')
sydney_merged.head()

,House,Year,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Elizabeth Farm, Sydney",1790s,Parramatta,-33.821191,151.017958,1,Hotel,Train Station,Gas Station,Pizza Place,Platform,Restaurant,Sandwich Place,Bowling Green,Supermarket,History Museum
1,"Vaucluse House, Sydney",1839,Vaucluse,-33.855789,151.273152,1,History Museum,Historic Site,Scenic Lookout,Café,Beach,Zoo,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,"Government House, Sydney",1830s,Sydney,-33.859925,151.215125,1,Café,Australian Restaurant,Hotel,Garden,Cocktail Bar,Harbor / Marina,Theater,Bar,Boat or Ferry,Concert Hall
3,"Admiralty House, Sydney",1840s,Kirribilli,-33.851728,151.218344,1,Boat or Ferry,Park,Harbor / Marina,Motel,Scenic Lookout,Pier,Café,Zoo,Fish & Chips Shop,Fast Food Restaurant
4,"Bronte House, Sydney",1845,Bronte,-33.904213,151.263777,1,Café,Beach,Juice Bar,Convenience Store,Restaurant,Park,Fish & Chips Shop,Burger Joint,Breakfast Spot,Pool


In [150]:
ddress = 'Sydney, AU'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are -33.8548157, 151.2164539.


In [151]:
import matplotlib.cm as cm
import matplotlib.colors as colors 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [152]:

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sydney_merged['Latitude'], sydney_merged['Longitude'], sydney_merged['Neighborhood'], sydney_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [153]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 0, sydney_merged.columns[[1] + list(range(5, sydney_merged.shape[1]))]]

,Year,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,1870s,0,Park,Boat or Ferry,Zoo,Food Truck,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
19,1840s,0,Park,Boat or Ferry,Zoo,Food Truck,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
24,1850s,0,Shopping Mall,Park,Zoo,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
28,1850s,0,Park,Zoo,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Dumpling Restaurant
36,1880s,0,Park,Pool Hall,Café,Creperie,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store


In [154]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 1, sydney_merged.columns[[1] + list(range(5, sydney_merged.shape[1]))]]

,Year,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1790s,1,Hotel,Train Station,Gas Station,Pizza Place,Platform,Restaurant,Sandwich Place,Bowling Green,Supermarket,History Museum
1,1839,1,History Museum,Historic Site,Scenic Lookout,Café,Beach,Zoo,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,1830s,1,Café,Australian Restaurant,Hotel,Garden,Cocktail Bar,Harbor / Marina,Theater,Bar,Boat or Ferry,Concert Hall
3,1840s,1,Boat or Ferry,Park,Harbor / Marina,Motel,Scenic Lookout,Pier,Café,Zoo,Fish & Chips Shop,Fast Food Restaurant
4,1845,1,Café,Beach,Juice Bar,Convenience Store,Restaurant,Park,Fish & Chips Shop,Burger Joint,Breakfast Spot,Pool
7,1790s,1,Indian Restaurant,Café,Pub,History Museum,Sports Bar,Park,South Indian Restaurant,Zoo,Fast Food Restaurant,Farmers Market
8,1790s,1,Football Stadium,Historic Site,Dentist's Office,Australian Restaurant,Tea Room,Bar,Golf Course,Park,Electronics Store,Flea Market
9,1810s,1,Café,Hotel,Australian Restaurant,Pub,Italian Restaurant,Scenic Lookout,Cocktail Bar,Ice Cream Shop,Park,Museum
10,1820s,1,Café,Bar,Park,Pizza Place,Coffee Shop,Restaurant,Italian Restaurant,Flea Market,Grocery Store,Lounge
11,1820s,1,Café,Italian Restaurant,Park,Pub,Hostel,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Fish & Chips Shop,Food Court


In [155]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 2, sydney_merged.columns[[1] + list(range(5, sydney_merged.shape[1]))]]

,Year,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,c. 1909,2,Fast Food Restaurant,Bus Station,Chinese Restaurant,Zoo,Electronics Store,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Farmers Market
46,1910s,2,Supermarket,Fast Food Restaurant,Convenience Store,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Farmers Market,Electronics Store,Eastern European Restaurant


In [156]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 3, sydney_merged.columns[[1] + list(range(5, sydney_merged.shape[1]))]]

,Year,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,1880s,3,Dog Run,Zoo,Eastern European Restaurant,Food Court,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store


In [157]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 4, sydney_merged.columns[[1] + list(range(5, sydney_merged.shape[1]))]]

,Year,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,1850s,4,Asian Restaurant,Spa,Zoo,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store


Results and Conclusion
  
In this study, we analyze our results according to the five clusters we have produced. Even though, all clusters has its own features, we have found two main patterns. The first pattern we are referring to Clusters 0 , may target people who want to go to a zoo after the visting. Instead, the second pattern we are referring to Clusters 1, may target individuals who love pubs, cafe. 